# Local Search strategy for the Knapsack problem

In [ ]:
import random

class Knapsack:
    def __init__(self, capacity, objects_values, objects_weights):
        self.capacity = capacity
        self.objects_values = objects_values
        self.objects_weights = objects_weights
        self.N = len(objects_values)

    def eval(self, S, alpha=0.5):
        weight = sum([a * b for a, b in zip(S, self.objects_weights)])
        value = sum([a * b for a, b in zip(S, self.objects_values)])

        if weight <= self.capacity:
            return -value
        else:
            return  (alpha*value + (1-alpha) * (self.capacity - weight))

    def init_solution(self):
        return [random.randrange(2) for i in range(self.N)]

    def get_random_neighbours(self, S):
        """
        Get random two-bit changed neighbours (works better)
        """
        neighbours = []

        for _ in range(30):
          neighbour = S.copy()
          first_index = random.randint(0, self.N - 1)
          second_index = random.randint(0, self.N - 1)

          neighbour[first_index] = 1 if neighbour[first_index] == 0 else 0
          neighbour[second_index] = 1 if neighbour[second_index] == 0 else 0

          neighbours.append(neighbour)

        return neighbours

    def get_all_neighbours(self, S):
        """
        Get all one-bit changed neighbours
        """
        neighbours = []

        for index in range(self.N):
          neighbour = S.copy()

          neighbour[index] = 1 if neighbour[index] == 0 else 0

          neighbours.append(neighbour)

        return neighbours

    def local_search(self, S):
        new_S = S.copy()

        while True:
          found_better_solution = False
          # Here if we use get_random_neighbours (generally) it will give a better solution
          neighbours = self.get_all_neighbours(new_S)
          print('-----------------')
          print(new_S)
          print('-----------------')
          for neighbour in neighbours:
            if self.eval(neighbour) > self.eval(new_S):
              new_S = neighbour
              found_better_solution = True

          if not found_better_solution:
            return new_S

In [ ]:
prob = Knapsack(20, [3,4,1,5,2], [10,8,3,6,5])

S = prob.init_solution()
S_eval = prob.eval(S)

print(S)
print(S_eval)
print()

S = prob.local_search(S)
S_eval = prob.eval(S)

print(S)
print(S_eval)
print()

[1, 0, 1, 1, 0]
-9

-----------------
[1, 0, 1, 1, 0]
-----------------
-----------------
[1, 0, 1, 1, 1]
-----------------
-----------------
[1, 0, 0, 1, 1]
-----------------
[1, 0, 0, 1, 1]
4.5



# Simulated Annealing for the Bin packing problem

In [ ]:
%matplotlib inline
import random, numpy, math, copy, matplotlib.pyplot as plt
import numpy as np

# acceptance probability
def accept(current_cost, new_cost, T):
    if new_cost <= current_cost:
        return True
    if random.random() < np.exp(-(new_cost-current_cost)/T) :
        return True
    else:
        return False

# cooling schedule
def updateTemperature(T, k):
    return  T*0.9995
    #return T/k

# Simulated Annealing algorithm
def SA(prob, T_Max, T_Min):
    S = prob.init_solution()
    S_cost = prob.eval(S)

    #prob.displayTour(S)

    S_best = S
    S_best_cost = S_cost

    T = T_Max
    k=0
    while T > T_Min:
        k=k+1
        S_prim = prob.perturb_solution(S)
        S_prim_cost = prob.eval(S_prim)

        if accept(S_cost, S_prim_cost, T):
            S = S_prim.copy()
            S_cost = S_prim_cost
        if S_cost < S_best_cost:
            S_best = S.copy()
            S_best_cost = S_cost
            print(S_best_cost, T)

        T = updateTemperature(T,k)

    return S_best_cost, S

In [ ]:
class Binpack:
    def __init__(self, objects_weights, bins_capacity):
        self.objects_weights = objects_weights
        self.bins_capacity = bins_capacity
        self.N = len(objects_weights)

    def eval(self, S):
        max_capacity = len(S) * self.bins_capacity
        ocupied_capacity = 0

        for bin in S:
          ocupied_capacity += sum(bin)

        return max_capacity - ocupied_capacity


    def init_solution(self):
        bin_index = 0
        bins = [[]]

        for object_weight in self.objects_weights:
          if (sum(bins[bin_index]) + object_weight > self.bins_capacity):
            bins.append([])
            bin_index += 1

          bins[bin_index].append(object_weight)

        return bins

    def swap(self, S):
        new_S = copy.deepcopy(S)

        first_bin_index = random.randint(0, len(S) - 1)
        first_object_index = random.randint(0, len(S[first_bin_index]) - 1)

        second_bin_index = random.randint(0, len(S) - 1)
        second_object_index = random.randint(0, len(S[second_bin_index]) - 1)

        aux = new_S[first_bin_index][first_object_index]
        new_S[first_bin_index][first_object_index] = new_S[second_bin_index][second_object_index]
        new_S[second_bin_index][second_object_index] = aux

        if (sum(new_S[first_bin_index]) > self.bins_capacity or sum(new_S[second_bin_index]) > self.bins_capacity):
          return S

        return new_S

    def move(self, S):
        new_S = copy.deepcopy(S)

        first_bin_index = random.randint(0, len(S) - 1)
        first_object_index = random.randint(0, len(S[first_bin_index]) - 1)

        second_bin_index = random.randint(0, len(S) - 1)

        new_S[second_bin_index].append(new_S[first_bin_index][first_object_index])
        new_S[first_bin_index].pop(first_object_index)

        if (sum(new_S[second_bin_index]) > self.bins_capacity):

          return S

        if (len(new_S[first_bin_index]) == 0):
          new_S.pop(first_bin_index)

        return new_S

    def perturb_solution(self, S):
        swap_or_move = random.randint(0, 1)

        return self.move(S) if swap_or_move == 0 else self.swap(S)


In [ ]:
prob = Binpack([4, 8, 1, 4, 2, 1], 10)

best, S = SA(prob, 1000., 0.000001)

print(best)
print(S)

0 994.0164725309133
0
[[1, 8, 1], [4, 2, 4]]


# Evolutionary Algorithm for the Bin packing problem

In [ ]:
import numpy as np

# Evolutionary Algorithm - overall structure
class Element:
    def __init__(self, encoding, cost=0):
        self.encoding = encoding
        self.cost = cost
    def __repr__(self):
        return "[el={}, cost={}]".format(self.encoding, self.cost)

    def copy(self):
        return Element(self.encoding.copy(), self.cost)

def populationInitialization(prob, popSize):
    population = [Element(prob.generate_random_elem()) for i in range(popSize)]
    return population

def populationEvaluation(prob, population):
    for el in population:
        el.cost = prob.eval(el.encoding)

def computeProbabilities(population):
    """
    Remark:  the selection probabilities are computed based on the adjusted cost (in order to be positive)
    """
    n = len(population)
    probabilities = []
    s = 0.0
    minCost = 0.0
    for el in population:
        if el.cost<minCost:
            minCost=el.cost
    for el in population:
        s += el.cost-minCost
    for el in population:
        probabilities.append((el.cost-minCost)/s)
    return probabilities

def proportionalSelection(probabilities):
    """
    Remark:  it returns an index generated using the cumulative probability distribution
    """
    k = 0
    s = probabilities[0]
    u = np.random.rand()
    while u > s:
        k += 1
        s = s + probabilities[k]
    return k

def rouletteWheel(probilities):
    """
    Remark: it generates the list of selected indices
    """
    n = len(probilities)
    indexes = []
    for i in range(n):
        indexes.append(proportionalSelection(probabilities))
    return indexes

def tournamentSelection(population, populationSize):
    """
    Remark: two randomly selected elements are sampled and the index of the best one is returned
    """
    i1 = np.random.randint(0, populationSize)
    i2 = np.random.randint(0, populationSize)
    if population[i1].cost < population[i2].cost:
        return i1
    else:
        return i2

def populationRecombination(recombinationFunc, prob, population, indexP1, indexP2):
    return getattr(prob, recombinationFunc)(population[indexP1].encoding, population[indexP2].encoding)

def new_population_build_byCopyChildren(population, children, limit=None):
    return children

def new_population_build_byParentsChildrenCompetition(population, children, value=None):
    popSize = len(population)
    newPop =[]
    for i in range(popSize):
        i1 = np.random.randint(0, 2*popSize)
        if i1 < popSize:
            el1 = population[i1]
        else:
            el1 = children[i1-popSize]

        i2 = np.random.randint(0, 2*popSize)
        if i2 < popSize:
            el2 = population[i2]
        else:
            el2 = children[i2-popSize]

        if el1.cost < el2.cost:
            newPop.append(el1.copy())
        else:
            newPop.append(el2.copy())
    return newPop

def EvolutionaryAlgorithm(prob, recombinationFunc=None, mutationFunc=None, popSize =10, pm=0.05, maxIter = 50,
                    new_population_build = new_population_build_byParentsChildrenCompetition, limit=None, selection='tournament'):

    population = populationInitialization(prob, popSize)
    populationEvaluation(prob, population)
    it = 0
    while it < maxIter:
        probabilities = computeProbabilities(population)
        # initialization of the population of children
        children = []
        # crossover
        for i in range(popSize//2):
            if selection == 'tournament':
                i1 = tournamentSelection(population, popSize)
                i2 = tournamentSelection(population, popSize)
            else:
                i1 = proportionalSelection(probabilities)
                i2 = proportionalSelection(probabilities)
            c1, c2 = populationRecombination(recombinationFunc, prob, population, i1, i2)
            children.extend([Element(c1), Element(c2)])

        # mutation
        for i in range(popSize):
            children[i].encoding = getattr(prob, mutationFunc)(children[i].encoding)
        # evaluation of the children
        populationEvaluation(prob, children)

        # population of survivors
        population = new_population_build(population, children, limit)
        # next generation
        it += 1
    # best element of the population
    best = min(population, key=lambda x: x.cost)
    return best

In [ ]:
import random

class Binpack:
    def __init__(self, capacity, objects_weights):
        self.capacity = capacity
        self.objects_weights = objects_weights
        self.N = len(objects_weights)

    def eval(self, S):
      bins_weights = [0] * max(S)

      for object_index in range(self.N):
        bins_weights[S[object_index] - 1] += self.objects_weights[object_index]

      unused_weights_in_bins = 0

      for bin_weight in bins_weights:
        # If the bin weight is 0 we don't calculate with because it means we can
        # discard the bin
        if bin_weight != 0:
          if bin_weight > self.capacity:
            unused_weights_in_bins += bin_weight
          else:
            unused_weights_in_bins += self.capacity - bin_weight

      return unused_weights_in_bins

    def generate_random_elem(self):
      objects_weights_copy = self.objects_weights.copy()
      solution = list(range(self.N))
      current_bin_index = 1
      current_bin_weight = 0
      not_all_objects_were_placed_in_bins = True

      while not_all_objects_were_placed_in_bins:
        random_object_index = random.randint(0, self.N - 1)
        random_object_weight = objects_weights_copy[random_object_index]

        # If the object weight is -1 it means we already put it in a bin
        if random_object_weight != -1:
          # If we can't put it in the current bin start a new empty one
          # else put it in the current bin
          if current_bin_weight + random_object_weight > self.capacity:
            current_bin_index += 1
            current_bin_weight = 0

          current_bin_weight += random_object_weight

          solution[random_object_index] = current_bin_index
          # Mark object as used (it was put in a bin)
          objects_weights_copy[random_object_index] = -1

        if sum(objects_weights_copy) == -1 * self.N:
          not_all_objects_were_placed_in_bins = False

      return solution

    def recombination_one_cut_point(self, el1, el2):
        # Get cutting point
        cutIndex = random.randint(0, self.N)

        child1 = el1[:cutIndex]
        child1 = child1 + el2[cutIndex:]

        child2 = el2[:cutIndex]
        child2 = child2 + el1[cutIndex:]

        return child1, child2

    def swap(self, S):
        S_copy = S.copy()

        first_object_index = random.randint(0, self.N - 1)
        second_object_index = random.randint(0, self.N - 1)

        aux = S_copy[first_object_index]
        S_copy[first_object_index] = S_copy[second_object_index]
        S_copy[second_object_index] = aux

        return S_copy

    def move(self, S):
        S_copy = S.copy()

        object_index = random.randint(0, self.N - 1)
        bin_to_move = random.randint(1, max(S))

        S_copy[object_index] = bin_to_move

        return S_copy

    def simple_mutation(self, S):
        swap_or_move = random.randint(0, 1)

        return self.move(S) if swap_or_move == 0 else self.swap(S)

In [ ]:
# EA applied for the Binpack problem
objects_weights = [4, 8, 1, 4, 2, 1]
prob = Binpack(10, objects_weights)
sol = EvolutionaryAlgorithm(prob, recombinationFunc='recombination_one_cut_point', mutationFunc='simple_mutation', popSize =20, pm=0.05,   maxIter = 150, selection='tournamentFeasibility')

print(objects_weights)
print(sol.encoding)

[4, 8, 1, 4, 2, 1]
[3, 1, 3, 3, 1, 3]
